<a href="https://colab.research.google.com/github/Diegoeverton/notebook_colaboratory/blob/main/Prevendo_fraudes_cartao_de_credito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Link you tube: https://www.youtube.com/watch?v=2Gf2pxB24Ic

LInk base de dados Kaggle: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud


Paradado video do you tube 20:00

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

In [4]:
df_credit = pd.read_csv("/content/drive/MyDrive/Base de dados/kaggle /Fraude cartao de credito/creditcard.csv")
df_credit.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
#Verfifica tipos
df_credit.dtypes

Time      float64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
Class       int64
dtype: object

In [6]:
# valores nulos
df_credit.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [7]:
# verifica os tipos dos dados nao fraude
df_nao_fraude = df_credit.Amount[df_credit.Class == 0]
df_nao_fraude.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [8]:
# verifica sostipos dos dados de fraude
df_fraude = df_credit.Amount[df_credit.Class == 1]
df_fraude.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [9]:
df_credit.Class.value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [10]:
# Verifica os tipos dos dados de fraude
df_fraude = df_credit[df_credit.Class == 1]
df_fraude

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
541,406.0,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,...,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,0.00,1
623,472.0,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,529.00,1
4920,4462.0,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,...,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,239.93,1
6108,6986.0,-4.397974,1.358367,-2.592844,2.679787,-1.128131,-1.706536,-3.496197,-0.248778,-0.247768,...,0.573574,0.176968,-0.436207,-0.053502,0.252405,-0.657488,-0.827136,0.849573,59.00,1
6329,7519.0,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,...,-0.379068,-0.704181,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,1.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1


In [11]:
# Verifica os tipos dos dados de não fraude
df_nao_fraude = df_credit[df_credit.Class == 0]
df_nao_fraude

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [12]:
# data set com fraude com a mesma quantidade de nao fraude
df_nao_fraudes = df_nao_fraude.sample (n=492)
df_nao_fraudes

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
10275,16158.0,1.277597,0.175731,-1.045243,0.273697,2.262720,3.432952,-0.531775,0.716960,1.137116,...,-0.128715,-0.241341,-0.139969,0.946950,0.807566,-0.304841,-0.007535,0.010217,12.00,0
71325,54256.0,1.355547,-0.520560,0.449579,-0.846560,-0.885825,-0.505036,-0.616777,-0.047603,-1.055315,...,-0.053568,-0.325528,0.062484,-0.005113,0.252806,-0.479839,0.005628,0.012746,25.00,0
159872,113014.0,2.127860,-0.103784,-1.746822,0.034944,0.451365,-0.560317,0.095935,-0.205635,0.920736,...,0.211110,0.720367,-0.059370,0.085899,0.414237,-0.409976,-0.002583,-0.055429,1.00,0
77717,57178.0,-1.778482,0.320782,1.657480,-3.459865,0.675439,1.168505,-0.434181,-0.963787,0.754350,...,1.824595,0.567145,-0.202696,-0.983535,0.345105,-0.037711,0.228367,0.008333,20.00,0
226915,144859.0,-0.838981,0.599675,0.133900,-1.710925,2.242866,-0.808341,1.678315,-0.385865,-0.881828,...,0.136568,0.334675,-0.699368,0.237712,1.347879,0.231342,-0.268231,-0.207182,7.70,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115488,73886.0,0.923619,-1.080508,0.465330,-0.473658,-1.210142,-0.665687,-0.255387,-0.197212,-0.993031,...,0.097361,-0.245695,0.001183,0.414674,0.070937,-0.498273,-0.006555,0.062518,225.00,0
193887,130341.0,-14.008848,-13.489450,-0.672586,2.763778,13.815072,-7.810116,-13.515145,-6.657748,3.230719,...,1.358282,-2.458450,-8.384873,1.217136,-3.087979,-0.244415,5.352193,-0.563544,0.76,0
281540,170250.0,-3.296875,2.924985,-2.237114,-0.999462,-0.247988,-1.252690,0.063705,0.683266,1.594078,...,0.046986,0.280853,-0.069211,0.613813,0.330438,0.590084,0.103159,0.648336,6.00,0
7586,10482.0,-0.718377,-0.152927,1.228138,1.156553,1.330884,0.657475,0.490095,-0.214661,1.131400,...,-0.128862,0.238470,0.107142,-1.247572,0.118459,-0.175890,-0.361508,-0.435178,57.57,0


In [13]:
# concatenar os dataframes nao fraude e fraude
df = pd.concat([df_nao_fraudes, df_fraude], axis = 0)
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
10275,16158.0,1.277597,0.175731,-1.045243,0.273697,2.262720,3.432952,-0.531775,0.716960,1.137116,...,-0.128715,-0.241341,-0.139969,0.946950,0.807566,-0.304841,-0.007535,0.010217,12.00,0
71325,54256.0,1.355547,-0.520560,0.449579,-0.846560,-0.885825,-0.505036,-0.616777,-0.047603,-1.055315,...,-0.053568,-0.325528,0.062484,-0.005113,0.252806,-0.479839,0.005628,0.012746,25.00,0
159872,113014.0,2.127860,-0.103784,-1.746822,0.034944,0.451365,-0.560317,0.095935,-0.205635,0.920736,...,0.211110,0.720367,-0.059370,0.085899,0.414237,-0.409976,-0.002583,-0.055429,1.00,0
77717,57178.0,-1.778482,0.320782,1.657480,-3.459865,0.675439,1.168505,-0.434181,-0.963787,0.754350,...,1.824595,0.567145,-0.202696,-0.983535,0.345105,-0.037711,0.228367,0.008333,20.00,0
226915,144859.0,-0.838981,0.599675,0.133900,-1.710925,2.242866,-0.808341,1.678315,-0.385865,-0.881828,...,0.136568,0.334675,-0.699368,0.237712,1.347879,0.231342,-0.268231,-0.207182,7.70,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1


In [14]:
# ajustar index
df.reset_index(inplace=True)
df

,index,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,10275,16158.0,1.277597,0.175731,-1.045243,0.273697,2.262720,3.432952,-0.531775,0.716960,...,-0.128715,-0.241341,-0.139969,0.946950,0.807566,-0.304841,-0.007535,0.010217,12.00,0
1,71325,54256.0,1.355547,-0.520560,0.449579,-0.846560,-0.885825,-0.505036,-0.616777,-0.047603,...,-0.053568,-0.325528,0.062484,-0.005113,0.252806,-0.479839,0.005628,0.012746,25.00,0
2,159872,113014.0,2.127860,-0.103784,-1.746822,0.034944,0.451365,-0.560317,0.095935,-0.205635,...,0.211110,0.720367,-0.059370,0.085899,0.414237,-0.409976,-0.002583,-0.055429,1.00,0
3,77717,57178.0,-1.778482,0.320782,1.657480,-3.459865,0.675439,1.168505,-0.434181,-0.963787,...,1.824595,0.567145,-0.202696,-0.983535,0.345105,-0.037711,0.228367,0.008333,20.00,0
4,226915,144859.0,-0.838981,0.599675,0.133900,-1.710925,2.242866,-0.808341,1.678315,-0.385865,...,0.136568,0.334675,-0.699368,0.237712,1.347879,0.231342,-0.268231,-0.207182,7.70,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
980,280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
981,280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
982,281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1


In [15]:
# dados de treinameto e teste + dados de validação
#returirar dados para validação
df_val_nao_fraude = df.head(5)
df_val_nao_fraude


,index,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,10275,16158.0,1.277597,0.175731,-1.045243,0.273697,2.262720,3.432952,-0.531775,0.716960,...,-0.128715,-0.241341,-0.139969,0.946950,0.807566,-0.304841,-0.007535,0.010217,12.0,0
1,71325,54256.0,1.355547,-0.520560,0.449579,-0.846560,-0.885825,-0.505036,-0.616777,-0.047603,...,-0.053568,-0.325528,0.062484,-0.005113,0.252806,-0.479839,0.005628,0.012746,25.0,0
2,159872,113014.0,2.127860,-0.103784,-1.746822,0.034944,0.451365,-0.560317,0.095935,-0.205635,...,0.211110,0.720367,-0.059370,0.085899,0.414237,-0.409976,-0.002583,-0.055429,1.0,0
3,77717,57178.0,-1.778482,0.320782,1.657480,-3.459865,0.675439,1.168505,-0.434181,-0.963787,...,1.824595,0.567145,-0.202696,-0.983535,0.345105,-0.037711,0.228367,0.008333,20.0,0
4,226915,144859.0,-0.838981,0.599675,0.133900,-1.710925,2.242866,-0.808341,1.678315,-0.385865,...,0.136568,0.334675,-0.699368,0.237712,1.347879,0.231342,-0.268231,-0.207182,7.7,0


In [16]:
df_val_fraude = df.tail(5)
df_val_fraude

,index,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
979,279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
980,280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
981,280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
982,281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1
983,281674,170348.0,1.991976,0.158476,-2.583441,0.408670,1.151147,-0.096695,0.223050,-0.068384,...,-0.164350,-0.295135,-0.072173,-0.450261,0.313267,-0.289617,0.002988,-0.015309,42.53,1


In [17]:
# retirar as 4 primeiras linhas
df = df.iloc[5:]

#retirar as 5 ultimas linhas
df = df[:-5]
df

,index,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
5,120785,75961.0,1.158020,-1.047580,1.388790,-0.559393,-1.981272,-0.275442,-1.329513,0.187483,...,-0.068804,0.118353,0.043924,0.575481,-0.137899,1.079356,-0.020119,0.028626,68.00,0
6,29804,35636.0,-0.412814,1.068848,1.566066,-0.035602,-0.054942,-0.879561,0.741081,-0.085885,...,-0.223412,-0.569099,-0.020821,0.369178,-0.181752,0.070340,0.266615,0.121722,5.68,0
7,270595,164155.0,2.197820,-0.827169,-1.637511,-1.169436,-0.190689,-0.492059,-0.498561,-0.135569,...,0.453422,1.203245,-0.038857,0.394582,0.218117,0.039263,-0.054448,-0.071947,25.00,0
8,97154,66089.0,-0.832326,0.501038,2.154949,0.918912,0.154125,1.275209,0.289983,0.294348,...,0.073315,0.813682,-0.156039,-0.243520,-0.045095,-0.199251,0.163080,-0.044399,43.05,0
9,123296,76874.0,-1.199846,0.961485,1.750827,0.123718,-0.820895,-0.288425,0.215216,0.687080,...,-0.181336,-0.619717,-0.022879,0.495009,0.239092,0.208124,0.112221,0.056849,49.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,274382,165981.0,-5.766879,-8.402154,0.056543,6.950983,9.880564,-5.773192,-5.748879,0.721743,...,0.880395,-0.130436,2.241471,0.665346,-1.890041,-0.120803,0.073269,0.583799,0.00,1
975,274475,166028.0,-0.956390,2.361594,-3.171195,1.970759,0.474761,-1.902598,-0.055178,0.277831,...,0.473211,0.719400,0.122458,-0.255650,-0.619259,-0.484280,0.683535,0.443299,39.90,1
976,275992,166831.0,-2.027135,-1.131890,-1.135194,1.086963,-0.010547,0.423797,3.790880,-1.155595,...,-0.315105,0.575520,0.490842,0.756502,-0.142685,-0.602777,0.508712,-0.091646,634.30,1
977,276071,166883.0,2.091900,-0.757459,-1.192258,-0.755458,-0.620324,-0.322077,-1.082511,0.117200,...,0.288253,0.831939,0.142007,0.592615,-0.196143,-0.136676,0.020182,-0.015470,19.95,1


In [18]:
# concatenar o data frame de validação
df_val_total = pd.concat([df_val_nao_fraude, df_val_fraude])

df_val_total.reset_index(inplace=True)
df_val_total_real = df_val_total.Class
df_val_total = df_val_total.drop(['level_0', 'index', 'Time', 'Class'], axis=1)
df_val_total

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,1.277597,0.175731,-1.045243,0.273697,2.262720,3.432952,-0.531775,0.716960,1.137116,-0.248344,...,-0.044583,-0.128715,-0.241341,-0.139969,0.946950,0.807566,-0.304841,-0.007535,0.010217,12.00
1,1.355547,-0.520560,0.449579,-0.846560,-0.885825,-0.505036,-0.616777,-0.047603,-1.055315,0.757489,...,0.145391,-0.053568,-0.325528,0.062484,-0.005113,0.252806,-0.479839,0.005628,0.012746,25.00
2,2.127860,-0.103784,-1.746822,0.034944,0.451365,-0.560317,0.095935,-0.205635,0.920736,-0.074288,...,-0.257070,0.211110,0.720367,-0.059370,0.085899,0.414237,-0.409976,-0.002583,-0.055429,1.00
3,-1.778482,0.320782,1.657480,-3.459865,0.675439,1.168505,-0.434181,-0.963787,0.754350,-1.972575,...,-0.747947,1.824595,0.567145,-0.202696,-0.983535,0.345105,-0.037711,0.228367,0.008333,20.00
4,-0.838981,0.599675,0.133900,-1.710925,2.242866,-0.808341,1.678315,-0.385865,-0.881828,-0.730440,...,0.055593,0.136568,0.334675,-0.699368,0.237712,1.347879,0.231342,-0.268231,-0.207182,7.70
5,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,-5.587794,...,1.252967,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00
6,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,-3.232153,...,0.226138,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76
7,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,-3.463891,...,0.247968,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89
8,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,-5.245984,...,0.306271,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00
9,1.991976,0.158476,-2.583441,0.408670,1.151147,-0.096695,0.223050,-0.068384,0.577829,-0.888722,...,-0.017652,-0.164350,-0.295135,-0.072173,-0.450261,0.313267,-0.289617,0.002988,-0.015309,42.53


In [19]:
from pandas._libs.hashtable import value_count
# Veriricar a distribuição dos dados do tipo fraude e nao fraude
df.Class.value_counts()

0    487
1    487
Name: Class, dtype: int64

In [20]:
# serpara labes par features
X = df.drop(['index', 'Time', 'Class'], axis=1)
Y = df['Class']

In [21]:
print (X, Y)

           V1        V2        V3        V4        V5        V6        V7  \
5    1.158020 -1.047580  1.388790 -0.559393 -1.981272 -0.275442 -1.329513   
6   -0.412814  1.068848  1.566066 -0.035602 -0.054942 -0.879561  0.741081   
7    2.197820 -0.827169 -1.637511 -1.169436 -0.190689 -0.492059 -0.498561   
8   -0.832326  0.501038  2.154949  0.918912  0.154125  1.275209  0.289983   
9   -1.199846  0.961485  1.750827  0.123718 -0.820895 -0.288425  0.215216   
..        ...       ...       ...       ...       ...       ...       ...   
974 -5.766879 -8.402154  0.056543  6.950983  9.880564 -5.773192 -5.748879   
975 -0.956390  2.361594 -3.171195  1.970759  0.474761 -1.902598 -0.055178   
976 -2.027135 -1.131890 -1.135194  1.086963 -0.010547  0.423797  3.790880   
977  2.091900 -0.757459 -1.192258 -0.755458 -0.620324 -0.322077 -1.082511   
978 -1.374424  2.793185 -4.346572  2.400731 -1.688433  0.111136 -0.922038   

           V8        V9       V10  ...       V20       V21       V22  \
5  

In [23]:
# se´para entre dados de treino e teste
X_train, X_test,Y_train, Y_test = train_test_split(X,Y,
                                                   test_size=0.2,
                                                   random_state=42,
                                                   stratify=Y)

In [27]:
# Treinamento

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train , Y_train)
pred = lr.predict(X_test)
acc = accuracy_score(Y_test, pred)


f'Acuracia {acc * 100:.2f}'

'Acuracia 93.33'

In [29]:
# Validação
pred = lr.predict(df_val_total)

df = pd.DataFrame({'real':df_val_total_real, 'previsao':pred})

print(df)

   real  previsao
0     0         0
1     0         0
2     0         0
3     0         0
4     0         0
5     1         1
6     1         1
7     1         1
8     1         1
9     1         0
